# Підготовка даних

## Файл csv із збігами та їх прізвищами

Мають бути такі колонки

- surname - прізвище предка збіженця. У чоловічому роді, українською мовою
- name - ім'я збіженця
- cm - число, спільна днк в сантиморганах
- url - покликання на збіженця

# Параметри

In [41]:
# Всі необхідні файли потрібно покласи у цей каталог
directory = "data"

# Файл із збіженцями та прізвищами їх предків
input_matches = "matches.csv"

multy = 1000

# Відкриваємо вхідний файл із збіженцями прізвищами їх предків та базу даних

In [20]:
import sqlite3

db = sqlite3.connect(f'{directory}/surnames.db')

In [13]:
import pandas as pd
matches_df = pd.read_csv(f'{directory}/matches.csv')
matches_df

,surname,group,name,cm,url
0,Шиліпук,NaN,Olya Shylipuk,41.7,https://www.myheritage.com.ua/dna/match/D-FE4A...
1,Барчук,NaN,Olya Shylipuk,41.7,https://www.myheritage.com.ua/dna/match/D-FE4A...
2,Шмаленко,NaN,Inna Shmalenko,40.5,https://www.myheritage.com.ua/dna/match/D-FE4A...
3,Купчик,NaN,Inna Shmalenko,40.5,https://www.myheritage.com.ua/dna/match/D-FE4A...
4,Полікарпенко,NaN,Александр Поликаренко,37.7,https://www.myheritage.com.ua/dna/match/D-FE4A...
...,...,...,...,...,...
102,Лащук,NaN,John Michalchuk,24.2,https://www.myheritage.com.ua/dna/match/D-FE4A...
103,Буз,M,Natalia Kardash,24.1,https://www.myheritage.com.ua/dna/match/D-FE4A...
104,Солодуха,M,Natalia Kardash,24.1,https://www.myheritage.com.ua/dna/match/D-FE4A...
105,Оселедько,NaN,Oksana Baškė,24.1,https://www.myheritage.com.ua/dna/match/D-FE4A...


# Підготовка результуючої таблиці

In [35]:
table_name = "distribution"
cursor = db.cursor()
cursor.execute(f'DROP TABLE IF EXISTS {table_name}')
cursor.execute(
    f'''
    CREATE TABLE {table_name} (
    	"region"	INTEGER NOT NULL,
    	"county"	INTEGER NOT NULL,
    	"city"	TEXT NOT NULL,
    	"score"	REAL NOT NULL,
    	PRIMARY KEY("region","county","city")
    )
    '''
)
db.commit()


In [33]:
match_grouped = matches_df.groupby(["url", "cm"]).surname

In [48]:
for (_, cm), surnames in match_grouped:
    cm_float = float(cm)
    str_surnames = "','".join(map(lambda t: t.upper(), surnames))
    select = f'''
        SELECT 
            COUNT(*) AS cn,
        	"Областьнародження",
        	IFNULL("Районнародження",0),
        	"Названаселеногопунктународження(Україна)"
        FROM people 
        where 
        	"Прізвищє" in ('{str_surnames}') 
        	AND "Стать"=1 
        	AND "Названаселеногопунктународження(Україна)" IS NOT NULL 
        	AND "Областьнародження" IS NOT NULL
        GROUP BY "Областьнародження",
        	"Районнародження",
        	"Названаселеногопунктународження(Україна)"    
    '''    

    cursor.execute(select)
    res = cursor.fetchall()
    total = 0
    for t in res:
        total = total + t[0]

    def update_params(row):
        (cn, region, county, city) = row
        score = (cm_float*cn*multy)/total
        return (region, county, city, score, score)

    cursor.executemany(
        f'''
        INSERT INTO {table_name}(region, county, city, score)
          VALUES(?, ?, ?, ?)
          ON CONFLICT(region, county, city) DO UPDATE SET
            score=score+?;
        ''',
        list(map(update_params, res))
    )
    db.commit()
    

In [49]:
db.close()